# Implementation

## Installation

In [ ]:
!pip install -q --upgrade transformers datasets tokenizers #huggingface
!pip install -q emoji pythainlp sklearn-pycrfsuite seqeval #thai2transformers
!rm -r thai2transformers thai2transformers_parent #ลบ folder ที่ download มากรณีรัน cell นี้ซ้ำ
!git clone -b dev https://github.com/vistec-AI/thai2transformers/
!mv thai2transformers thai2transformers_parent
!mv thai2transformers_parent/thai2transformers .
!pip install accelerate==0.5.1
!apt install git-lfs
!pip install sentencepiece
!pip install huggingface_hub

     |████████████████████████████████| 4.2 MB 7.9 MB/s 
     |████████████████████████████████| 346 kB 52.1 MB/s 
     |████████████████████████████████| 6.6 MB 45.2 MB/s 
     |████████████████████████████████| 86 kB 4.2 MB/s 
     |████████████████████████████████| 596 kB 59.5 MB/s 
     |████████████████████████████████| 1.1 MB 46.5 MB/s 
     |████████████████████████████████| 212 kB 60.0 MB/s 
     |████████████████████████████████| 140 kB 55.3 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
     |████████████████████████████████| 127 kB 58.1 MB/s 
     |████████████████████████████████| 144 kB 63.8 MB/s 
     |████████████████████████████████| 94 kB 1.0 MB/s 
     |████████████████████████████████| 271 kB 57.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is 

## Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
from datasets import load_dataset, load_metric, DatasetDict, Dataset, load_from_disk
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import EarlyStoppingCallback
from thai2transformers.preprocess import process_transformers
from thai2transformers.metrics import classification_metrics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Loading the Dataset

In [ ]:
split_dataset = load_from_disk('/content/drive/MyDrive/Fake news/News-Dataset/augmented_dataset')

In [ ]:
def clean_function(examples):
    examples['text'] = process_transformers(examples['text'])
    return examples

cleaned_dataset = split_dataset.map(clean_function)

  0%|          | 0/4860 [00:00<?, ?ex/s]

  0%|          | 0/676 [00:00<?, ?ex/s]

  0%|          | 0/270 [00:00<?, ?ex/s]

In [ ]:
cleaned_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', '__index_level_0__'],
        num_rows: 4860
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 676
    })
    valid: Dataset({
        features: ['text', 'labels'],
        num_rows: 270
    })
})

In [ ]:
pd.DataFrame(cleaned_dataset['train'].shuffle()[:100])[['labels','text']]

,labels,text
0,1,ชายอังกฤษเกือบตายเพราะเยื่อบุหัวใจอักเสบหลังแค...
1,1,หน้ากากอนามัยคนไม่ป่วยใส่ช่วยกันเชื้อไม่ได้นะค...
2,1,ชาวบ้านแห่เก็บต้นอังกาบหนูเชื่อรักษามะเร็งมี14...
3,1,6<_>เครื่องดื่มที่ช่วย<_>vanquish<_>ภายใต้การค...
4,1,slamic<_>..<_>บำรุงหัวใจ<_>!!<_>|||<_>สแลงด้วย...
...,...,...
95,0,เคล็ดลับ15ประการพิชิตสุขภาพดีเริ่มต้นได้ง่ายๆต...
96,1,แทบไม่น่าเชื่อลดน้ำหนักด้วยการดื่มน้ำลดได้ถึง4...
97,1,รู้เอาไว้คนที่เป็น2โรค“ห้ามดื่มน้ำมะพร้าว”|||น...
98,1,สรรคุณแกงถุงนี้มีสมุนไพรนับ100ชนิดรักษาคนเป็นอ...


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased')
tokenizer(cleaned_dataset['train'][0]['text'])

Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/883k [00:00<?, ?B/s]

{'input_ids': [5, 10, 19775, 303, 86, 732, 4973, 143, 4098, 33, 1979, 972, 23370, 5252, 18496, 19775, 303, 86, 732, 4973, 143, 4098, 33, 1979, 972, 23370, 303, 86, 732, 4973, 143, 4098, 33, 1979, 972, 23370, 7173, 94, 3268, 185, 84, 2040, 52, 21, 142, 303, 86, 732, 4973, 143, 4098, 268, 303, 86, 732, 32, 4973, 15470, 13610, 207, 59, 303, 4178, 598, 21, 5198, 21392, 7196, 16691, 76, 22, 3620, 33, 143, 510, 207, 22081, 140, 6018, 1258, 280, 143, 4090, 4535, 3817, 2754, 59, 303, 21184, 148, 125, 356, 393, 17, 14876, 1258, 22, 111, 75, 56, 9, 1258, 3442, 4155, 37, 2556, 533, 3189, 96, 56, 17839, 3313, 13390, 2672, 56, 9, 44, 5611, 3620, 1258, 21184, 148, 125, 1178, 204, 4898, 23370, 11993, 13, 796, 15, 151, 3620, 1258, 15, 2221, 30, 2110, 83, 974, 176, 709, 30, 15618, 563, 14876, 3620, 1258, 64, 125, 51, 2556, 1178, 382, 4898, 2672, 15762, 3159, 75, 56, 9, 103, 1979, 44, 78, 1051, 3620, 1258, 1766, 1979, 2727, 1979, 2235, 890, 17, 1108, 906, 197, 17, 14876, 3278, 99, 159, 1289, 118, 6084, 

In [ ]:
def encode_function(examples):
    return tokenizer(examples['text'], max_length=416, truncation=True)
encoded_dataset = cleaned_dataset.map(encode_function, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Fine-tuning

In [ ]:
num_labels = len(set(encoded_dataset['train']['labels']))

model = AutoModelForSequenceClassification.from_pretrained(
    'airesearch/wangchanberta-base-att-spm-uncased', 
    num_labels=num_labels)

Downloading:   0%|          | 0.00/404M [00:00<?, ?B/s]

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wa

In [ ]:
#setup trainer
train_args = TrainingArguments(
    output_dir = 'wisesight_sentiment_wangchanberta',
    evaluation_strategy = "steps",
    eval_steps = 50,
    learning_rate=5e-05,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=20,
    warmup_steps = int(len(encoded_dataset['train']) * 1 // 8 * 0.1),
    weight_decay=1e-2,
    save_total_limit=5,
    metric_for_best_model='f1_micro',
    seed = 125,
    load_best_model_at_end=True
)

trainer = Trainer(
    model,
    train_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['valid'],
    tokenizer=tokenizer,
    compute_metrics=classification_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=7)]
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `CamembertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4860
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 12160


Step,Training Loss,Validation Loss,Accuracy,F1 Micro,Precision Micro,Recall Micro,F1 Macro,Precision Macro,Recall Macro,Nb Samples
50,No log,0.551661,0.762963,0.762963,0.762963,0.762963,0.750649,0.823047,0.761359,270
100,No log,0.337983,0.874074,0.874074,0.874074,0.874074,0.873901,0.875496,0.873848,270
150,No log,0.563484,0.814815,0.814815,0.814815,0.814815,0.811831,0.838529,0.815792,270
200,No log,0.428685,0.855556,0.855556,0.855556,0.855556,0.855554,0.855556,0.855575,270
250,No log,0.418822,0.866667,0.866667,0.866667,0.866667,0.866637,0.866765,0.866604,270
300,No log,0.611016,0.811111,0.811111,0.811111,0.811111,0.811088,0.811494,0.811238,270
350,No log,0.371388,0.859259,0.859259,0.859259,0.859259,0.859190,0.859615,0.859142,270
400,No log,0.505346,0.844444,0.844444,0.844444,0.844444,0.844368,0.845566,0.844655,270
450,No log,0.572684,0.800000,0.800000,0.800000,0.800000,0.799295,0.805308,0.800483,270
500,0.416600,0.587004,0.800000,0.800000,0.800000,0.800000,0.797500,0.817597,0.800867,270


The following columns in the evaluation set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `CamembertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 270
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `CamembertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 270
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `CamembertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 270
  Batc

TrainOutput(global_step=2350, training_loss=0.2766045850388547, metrics={'train_runtime': 1064.6855, 'train_samples_per_second': 91.295, 'train_steps_per_second': 11.421, 'total_flos': 4010585372411520.0, 'train_loss': 0.2766045850388547, 'epoch': 3.87})

## Test Results

In [ ]:
preds  = trainer.predict(encoded_dataset['test'])
pd.DataFrame.from_dict(preds[2],orient='index')

The following columns in the test set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `CamembertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 676
  Batch size = 8


,0
test_loss,0.462254
test_accuracy,0.908284
test_f1_micro,0.908284
test_precision_micro,0.908284
test_recall_micro,0.908284
test_f1_macro,0.907556
test_precision_macro,0.906822
test_recall_macro,0.908549
test_nb_samples,676.000000
test_runtime,9.957800


In [ ]:
import torch
torch.tensor(preds[0])
probabilities = torch.nn.functional.softmax(torch.tensor(preds[0]), dim=-1)
predictions = torch.argmax(probabilities, axis=1)
predictions
from sklearn.metrics import classification_report
y_true = encoded_dataset['test']['labels']
y_pred = preds[1]
target_names = ['fact news', 'fake news']
print(classification_report(y_true, predictions, target_names=target_names))

              precision    recall  f1-score   support

   fact news       0.89      0.91      0.90       304
   fake news       0.93      0.91      0.92       372

    accuracy                           0.91       676
   macro avg       0.91      0.91      0.91       676
weighted avg       0.91      0.91      0.91       676



# Upload to Huggingface Hub

In [ ]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model.push_to_hub("soda-berta")